### Lab 2. QA/QC on Datasets

In [1]:
import arcpy
import numpy as np
import pandas as pd

import io
import os
import sys
import json

### Part 2. Clean and Prepare the Data

In [2]:
# Verify the integrity of each file
file_paths = {
    "NLCD": r"C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\nlcd_mn_2019\NLCD_2019_Land_Cover.tif",
    "BMSB": r"C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\brown_marmorated_stink_bug_mn.csv",
    "DEM": r"C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\30m_mn_elev\elev_30m_digital_elevation_model.gdb"
}

for file_name, path in file_paths.items():
    if os.path.exists(path):
        print(f"{file_name} file found.")
    else:
        print(f"{file_name} file missing!")

NLCD file found.
BMSB file found.
DEM file found.


### Initial Assessment of Data

#### NCLD Data

In [23]:
with arcpy.EnvManager(outputCoordinateSystem='PROJCS["NAD_1983_UTM_Zone_15N",GEOGCS["GCS_North_American_1983",DATUM["D_North_American_1983",SPHEROID["GRS_1980",6378137.0,298.257222101]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",500000.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",-93.0],PARAMETER["Scale_Factor",0.9996],PARAMETER["Latitude_Of_Origin",0.0],UNIT["Meter",1.0]]'):
    arcpy.management.Clip(
        in_raster="NLCD_2019_Land_Cover.tif",
        rectangle="190687.908114248 4816425.97098429 758772.057946726 5471006.46677986",
        out_raster=r"C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\MyProject2.gdb\NLCD_2019_Clip",
        in_template_dataset="USA States Generalized selection",
        nodata_value="255",
        clipping_geometry="ClippingGeometry",
        maintain_clipping_extent="MAINTAIN_EXTENT"
    )
print("Clipping Completed!")

Clipping Completed!


In [24]:
# Check raster
nlcd_raster = r"C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\MyProject2.gdb\NLCD_2019_Clip"
desc = arcpy.Describe(nlcd_raster)
print(f"Raster Path: {desc.path}")
print(f"Raster Spatial Reference: {desc.spatialReference.name}")
print(f"Raster Size: {desc.width} x {desc.height}")

# Check for no data values
no_data_raster = arcpy.sa.IsNull(nlcd_raster)

# Use the RasterToNumPyArray tool to convert the raster to an array
no_data_array = arcpy.RasterToNumPyArray(no_data_raster)

# Count the number of NoData cells (where the value is 1)
no_data_count = np.count_nonzero(no_data_array == 1)
print(f"Count of NoData cells: {no_data_count}")

Raster Path: C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\MyProject2.gdb
Raster Spatial Reference: NAD_1983_UTM_Zone_15N
Raster Size: 18937 x 21820
Count of NoData cells: 170029673


#### DEM Data

In [25]:
# Check the DEM dataset
dem_gdb_path = r"C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\30m_mn_elev\elev_30m_digital_elevation_model.gdb"
dem_dataset = r"digital_elevation_model_30m"

try:
    dem_desc = arcpy.Describe(os.path.join(dem_gdb_path, dem_dataset))
    print(f"DEM Dataset Path: {dem_desc.path}")
    print(f"DEM Spatial Reference: {dem_desc.spatialReference.name}")
    print(f"DEM Size: {dem_desc.width} x {dem_desc.height}")
except Exception as e:
    print(f"Error describing DEM: {e}")

DEM Dataset Path: C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\30m_mn_elev\elev_30m_digital_elevation_model.gdb
DEM Spatial Reference: NAD_1983_UTM_Zone_15N
DEM Size: 19063 x 21871


#### Check Raster Data Consistency

In [26]:
# Check coordinate system consistency between NLCD and DEM
nlcd_sr = arcpy.Describe(nlcd_raster).spatialReference
dem_sr = arcpy.Describe(os.path.join(dem_gdb_path, dem_dataset)).spatialReference

if nlcd_sr.name == dem_sr.name:
    print("Coordinate systems match.")
else:
    print(f"Mismatch: NLCD SR = {nlcd_sr.name}, DEM SR = {dem_sr.name}")

Coordinate systems match.


In [27]:
# Load the CSV for inspection
bmsb_path = r"C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\brown_marmorated_stink_bug_mn_all_observations.csv"
try:
    bmsb_data = pd.read_csv(bmsb_path)
    print("CSV loaded successfully.")
    print("Columns:", bmsb_data.columns)
    print("First few rows:", bmsb_data.head())
except Exception as e:
    print(f"Error loading CSV: {e}")

# Check for missing or incorrect values
missing_values = bmsb_data.isnull().sum()
print("Missing values in columns:", missing_values)

CSV loaded successfully.
Columns: Index(['ID', 'Observed Date', 'Latitude', 'Longitude', 'Scientific Name',
       'Common Name', 'Image URL', 'Location Description', 'Exact Location?',
       'Obscured?', 'Accuracy (meters)', 'Count Observed'],
      dtype='object')
First few rows:           ID Observed Date   Latitude  Longitude    Scientific Name  \
0  264596414    2025-03-08  44.943124 -93.124529  Halyomorpha halys   
1  263160449    2024-01-18  44.010008 -92.456489  Halyomorpha halys   
2  262052201    2025-02-17  44.010681 -92.505242  Halyomorpha halys   
3  261700994    2025-02-14  44.895863 -93.262520  Halyomorpha halys   
4  260707725    2025-02-05  46.573231 -90.929597  Halyomorpha halys   

                  Common Name  \
0  Brown Marmorated Stink Bug   
1  Brown Marmorated Stink Bug   
2  Brown Marmorated Stink Bug   
3  Brown Marmorated Stink Bug   
4  Brown Marmorated Stink Bug   

                                           Image URL  \
0  https://inaturalist-open-data.s

### Assess Specific Data Aspects

In [4]:
!pip install rasterio

Defaulting to user installation because normal site-packages is not writeable


In [7]:
# Set your workspace
arcpy.env.workspace = "C:\\Users\\ethan\\Documents\\ArcGIS\\Projects\\MyProject2"

# Define paths for your datasets
dem_path = "C:\\Users\\ethan\\Documents\\ArcGIS\\Projects\\MyProject2\\30m_mn_elev\\elev_30m_digital_elevation_model.gdb\\digital_elevation_model_30m"
nlcd_path = "C:\\Users\\ethan\\Documents\\ArcGIS\\Projects\\MyProject2\\nlcd_mn_2019\\NLCD_2019_Land_Cover.tif"
bmsb_csv_path = "C:\\Users\\ethan\\Documents\\ArcGIS\\Projects\\MyProject2\\brown_marmorated_stink_bug_mn.csv"

# 1) Check for NoData
def check_nodata(raster_path):
    raster = arcpy.Raster(raster_path)
    band = arcpy.RasterToNumPyArray(raster)
    
    # Get NoData value from raster properties
    nodata_value = raster.noDataValue
    if nodata_value is not None:
        # Count NoData cells
        no_data_cells = np.sum(band == nodata_value)
        print(f"Number of NoData cells in {raster_path}: {no_data_cells}")
    else:
        print(f"No NoData value found for {raster_path}")

# 2) Verify Data Projection (NAD83 15N - EPSG: 26915)
def check_projection(raster_path):
    desc = arcpy.Describe(raster_path)
    crs = desc.spatialReference
    if crs.factoryCode == 26915:
        print(f"{raster_path} is in NAD83 UTM Zone 15N projection (EPSG: 26915).")
    else:
        print(f"{raster_path} is not in NAD83 UTM Zone 15N projection. It is in {crs.name} (EPSG: {crs.factoryCode}).")

# 3) Max/Min Check (DEM and NLCD)
def check_min_max(raster_path, expected_min, expected_max):
    raster = arcpy.Raster(raster_path)
    band = arcpy.RasterToNumPyArray(raster)
    min_value = np.nanmin(band)
    max_value = np.nanmax(band)
    if min_value < expected_min or max_value > expected_max:
        print(f"Out of range values detected in {raster_path}: Min={min_value}, Max={max_value}")
    else:
        print(f"{raster_path} is within expected range: Min={min_value}, Max={max_value}")

# 4) Duplicates Check (DEM and NLCD)
def check_duplicates(raster_path):
    raster = arcpy.Raster(raster_path)
    band = arcpy.RasterToNumPyArray(raster)
    unique_values, counts = np.unique(band, return_counts=True)
    duplicate_count = np.sum(counts > 1)
    print(f"Unique values in {raster_path}: {unique_values}")
    print(f"Duplicate cells in {raster_path}: {duplicate_count}")

# 5) Outlier Check (DEM)
def check_outliers(raster_path, threshold=3):
    raster = arcpy.Raster(raster_path)
    band = arcpy.RasterToNumPyArray(raster)
    
    mean = np.mean(band)
    std_dev = np.std(band)
    
    # Identify outliers using integer-safe comparison
    outliers = np.where((band > mean + threshold * std_dev) | (band < mean - threshold * std_dev))
    print(f"Outliers detected in {raster_path}: {len(outliers[0])} outlier cells")

# 6) CSV: Remove Invalid or Missing Data
def clean_csv(csv_path):
    df = pd.read_csv(csv_path)
    initial_row_count = len(df)
    df_cleaned = df.dropna()  # Drop rows with any missing data
    final_row_count = len(df_cleaned)
    print(f"Removed {initial_row_count - final_row_count} rows with missing data from {csv_path}")
    
    # Optionally, drop columns with missing data
    df_cleaned = df_cleaned.dropna(axis=1)
    return df_cleaned

# Running the full QA/QC pipeline for DEM, NLCD, and CSV
def run_qaqc_pipeline():
    # Run QA/QC for DEM
    print(f"\nRunning QA/QC for DEM at {dem_path}...\n")
    check_nodata(dem_path)
    check_projection(dem_path)
    check_min_max(dem_path, expected_min=-100, expected_max=5000)  # Example range for DEM
    check_duplicates(dem_path)
    check_outliers(dem_path)

    # Run QA/QC for NLCD
    print(f"\nRunning QA/QC for NLCD at {nlcd_path}...\n")
    check_nodata(nlcd_path)
    check_projection(nlcd_path)
    check_min_max(nlcd_path, expected_min=0, expected_max=100)  # Example range for NLCD
    check_duplicates(nlcd_path)

    # Clean and check the CSV data for BMSB
    print(f"\nCleaning CSV data from {bmsb_csv_path}...\n")
    cleaned_csv = clean_csv(bmsb_csv_path)
    cleaned_csv_filename = bmsb_csv_path.replace(".csv", "_cleaned.csv")
    cleaned_csv.to_csv(cleaned_csv_filename, index=False)
    print(f"Cleaned CSV saved to {cleaned_csv_filename}")

# Example Usage
run_qaqc_pipeline()


Running QA/QC for DEM at C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\30m_mn_elev\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m...

No NoData value found for C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\30m_mn_elev\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m
C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\30m_mn_elev\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m is in NAD83 UTM Zone 15N projection (EPSG: 26915).
C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\30m_mn_elev\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m is within expected range: Min=0, Max=2300
Unique values in C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\30m_mn_elev\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m: [   0  590  591 ... 2293 2297 2300]
Duplicate cells in C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\30m_mn_elev\elev_30m_digital_elevation_model.gdb\digital_elevation_model_30m: 150

#### Convert Rasters to Point

##### NLCD to Point

In [2]:
# Convert the Raster Files to Points and Join the Fields from the Raster
import arcpy
import os

# Set environment
arcpy.env.workspace = r"C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\output.gdb"
arcpy.env.overwriteOutput = True

# Input raster and output paths
input_raster = r"C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\MyProject2.gdb\NLCD_2019_Clip"
resampled_raster = "nlcd_resampled_1km"  # Name of the resampled raster
output_points = "nlcd_points_with_attributes_1km"

# RESAMPLE TO 1KM
# Set the cell size to 1000 meters (1 km)
cell_size = 1000
arcpy.Resample_management(
    input_raster, 
    resampled_raster, 
    cell_size, 
    "NEAREST"  # Nearest neighbor preserves categorical values
)
print(f"✅ Raster resampled to 1 km successfully: {resampled_raster}")

✅ Raster resampled to 1 km successfully: nlcd_resampled_1km


In [11]:
# Force rebuild attribute table to ensure consistency
desc = arcpy.Describe(resampled_raster)
pixel_type = desc.pixelType  # Get the pixel type (e.g., 8_BIT_UNSIGNED, 32_BIT_FLOAT, etc.)

# If the pixel type is integer (e.g., 8_BIT_UNSIGNED), it can have an attribute table
if "SIGNED" in pixel_type or "UNSIGNED" in pixel_type:
    print(f"✅ Raster is integer type: {pixel_type}. Ready to proceed with attribute table.")
else:
    print(f"⚠️ Raster is not an integer type: {pixel_type}. Consider reclassifying it.")

⚠️ Raster is not an integer type: U8. Consider reclassifying it.


In [12]:
# Rebuild the attribute table if it's missing or if you're unsure
arcpy.BuildRasterAttributeTable_management(resampled_raster, "OVERWRITE")

<Result 'nlcd_resampled_1km'>

In [13]:
# Join fields from the raster to the point feature class
fields_to_join = ["Count", "Red", "Green", "Blue", "Opacity", "NLCD_Land"]

arcpy.JoinField_management(
    output_points,
    "GRID_CODE",  # Point 'Value' field
    resampled_raster,  # Raster table to join from
    "Value",  # Matching field in the raster
    fields_to_join  # Fields to join
)

print(f"✅ Successfully joined fields: {fields_to_join}")

✅ Successfully joined fields: ['Count', 'Red', 'Green', 'Blue', 'Opacity', 'NLCD_Land']


In [14]:
# CONVERT RASTER TO POINTS
# Convert resampled raster to points
arcpy.RasterToPoint_conversion(resampled_raster, output_points, "VALUE")
print("✅ Resampled raster converted to points successfully!")

✅ Resampled raster converted to points successfully!


In [17]:
# List available fields in raster attribute table
point_fields = [f.name for f in arcpy.ListFields(output_points)]
print(f"🎨 Available point fields: {point_fields}")

🎨 Available point fields: ['OBJECTID', 'Shape', 'pointid', 'grid_code']


In [16]:
# JOIN ALL ATTRIBUTES TO POINTS
try:
    print("🔄 Building attribute table if needed...")

    # Check raster type (integer or floating-point)
    desc = arcpy.Describe(resampled_raster)
    pixel_type = desc.pixelType

    if "SIGNED" in pixel_type or "UNSIGNED" in pixel_type:  # Check if integer
        # Build the attribute table if it's missing
        arcpy.management.BuildRasterAttributeTable(resampled_raster, "OVERWRITE")
        print("✅ Attribute table built successfully!")
    else:
        print(f"⚠️ Raster is not integer-based (pixel type: {pixel_type}). Attribute table cannot be built.")
    
except Exception as e:
    print(f"⚠️ Attribute table could not be built: {e}")

# Check if attribute table exists
if "SIGNED" in pixel_type or "UNSIGNED" in pixel_type:
    # Proceed with the join
    has_attribute_table = True
    attribute_table = resampled_raster
    print(f"📝 Attribute table found. Ready to join fields!")
else:
    has_attribute_table = False
    print("⚠️ No attribute table found. Skipping join.")

# Join Attributes to Points
if has_attribute_table:
    fields_to_join = [
        f.name for f in arcpy.ListFields(attribute_table)
        if f.name not in ("OBJECTID", "Value", "GRID_CODE", "Shape")
    ]

    if fields_to_join:
        arcpy.JoinField_management(
            output_points,
            "GRID_CODE",  # Point 'Value' field
            attribute_table,
            "Value",  # Matching field in the raster
            fields_to_join
        )
        print(f"✅ Successfully joined fields: {fields_to_join}")
    else:
        print("⚠️ No additional fields to join.")
else:
    print("⚠️ Skipping attribute join due to missing attribute table.")

print(f"🎉 Points with 1 km resampled data and full attributes saved at: {os.path.join(arcpy.env.workspace, output_points)}")

🔄 Building attribute table if needed...
⚠️ Raster is not integer-based (pixel type: U8). Attribute table cannot be built.
⚠️ No attribute table found. Skipping join.
⚠️ Skipping attribute join due to missing attribute table.
🎉 Points with 1 km resampled data and full attributes saved at: C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\output.gdb\nlcd_points_with_attributes_1km


##### DEM to Point

In [1]:
# Convert the Raster Files to Points and Join the Fields from the Raster
import arcpy
import os

# Set environment
arcpy.env.workspace = r"C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\output.gdb"
arcpy.env.overwriteOutput = True

# Input raster and output paths
input_raster = "C:\\Users\\ethan\\Documents\\ArcGIS\\Projects\\MyProject2\\30m_mn_elev\\elev_30m_digital_elevation_model.gdb\\digital_elevation_model_30m"
resampled_raster = "dem_resampled_1km"  # Name of the resampled raster
output_points = "dem_points_with_attributes_1km"

# RESAMPLE TO 1KM
# Set the cell size to 1000 meters (1 km)
cell_size = 1000
arcpy.Resample_management(
    input_raster, 
    resampled_raster, 
    cell_size, 
    "NEAREST"  # Nearest neighbor preserves categorical values
)
print(f"✅ Raster resampled to 1 km successfully: {resampled_raster}")

✅ Raster resampled to 1 km successfully: dem_resampled_1km


In [2]:
# Force rebuild attribute table to ensure consistency
desc = arcpy.Describe(resampled_raster)
pixel_type = desc.pixelType  # Get the pixel type (e.g., 8_BIT_UNSIGNED, 32_BIT_FLOAT, etc.)

# If the pixel type is integer (e.g., 8_BIT_UNSIGNED), it can have an attribute table
if "SIGNED" in pixel_type or "UNSIGNED" in pixel_type:
    print(f"✅ Raster is integer type: {pixel_type}. Ready to proceed with attribute table.")
else:
    print(f"⚠️ Raster is not an integer type: {pixel_type}. Consider reclassifying it.")

⚠️ Raster is not an integer type: S16. Consider reclassifying it.


In [3]:
# Rebuild the attribute table if it's missing or if you're unsure
arcpy.BuildRasterAttributeTable_management(resampled_raster, "OVERWRITE")

<Result 'dem_resampled_1km'>

In [4]:
# CONVERT RASTER TO POINTS
# Convert resampled raster to points
arcpy.RasterToPoint_conversion(resampled_raster, output_points, "VALUE")
print("✅ Resampled raster converted to points successfully!")

✅ Resampled raster converted to points successfully!


### Part 3: Upload Data to Cloud Database

In [ ]:
C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\brown_marmorated_stink_bug_mn_cleaned.
{cleaned_csv_filename}

In [ ]:
# Use FeatureClassToFeatureClass to move data to PostgreSQL
arcpy.FeatureClassToFeatureClass_conversion(
    in_features="nlcd_points_with_attributes_1km",  # Input feature class
    out_path=r"C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\PostgreSQL-34-lab0(postgres).sde",  # Output database path
    out_name="nlcd_points",  # Name of the feature class in the database
    where_clause="",  # Optional: add a WHERE clause for filtering
    field_mapping='pointid "pointid" true true false 4 Long 0 0,First,#,nlcd_points_with_attributes_1km,pointid,-1,-1;grid_code "grid_code" true true false 4 Long 0 0,First,#,nlcd_points_with_attributes_1km,grid_code,-1,-1'
)

print("✅ Feature class successfully exported to PostgreSQL!")

In [ ]:
# Use FeatureClassToFeatureClass to move data to PostgreSQL
arcpy.FeatureClassToFeatureClass_conversion(
    in_features="dem_points_with_attributes_1km",  # Input feature class
    out_path=r"C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\PostgreSQL-34-lab0(postgres).sde",  # Output database path
    out_name="dem_points",  # Name of the feature class in the database
    where_clause="",  # Optional: add a WHERE clause for filtering
    field_mapping='pointid "pointid" true true false 4 Long 0 0,First,#,dem_points_with_attributes_1km,pointid,-1,-1;grid_code "grid_code" true true false 4 Long 0 0,First,#,dem_points_with_attributes_1km,grid_code,-1,-1'
)

print("✅ Feature class successfully exported to PostgreSQL!")

In [1]:
# Convert CSV to Point
arcpy.management.XYTableToPoint(
    in_table=r"C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\brown_marmorated_stink_bug_mn.csv",
    out_feature_class=r"C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\MyProject2.gdb\brown_marmorated_stink_bug_mn_XYTableToPoint",
    x_field="Longitude",
    y_field="Latitude",
    z_field=None,
    coordinate_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]];-400 -400 1000000000;-100000 10000;-100000 10000;8.98315284119521E-09;0.001;0.001;IsHighPrecision'
)

<Result 'C:\\Users\\ethan\\Documents\\ArcGIS\\Projects\\MyProject2\\MyProject2.gdb\\brown_marmorated_stink_bug_mn_XYTableToPoint'>

In [3]:
# Set the connection to your PostgreSQL database (make sure you have an SDE connection file)
sde_connection = r"C:\Users\ethan\Documents\ArcGIS\Projects\MyProject2\PostgreSQL-34-lab0(postgres).sde"

# Define the output feature class name in the PostgreSQL database
output_feature_class = "observations.bmsb_mn_XYTableToPoint"

# Export features directly to PostgreSQL/PostGIS
arcpy.FeatureClassToFeatureClass_conversion(
    in_features="brown_marmorated_stink_bug_mn_XYTableToPoint",  # Input feature class
    out_path=sde_connection,  # Connection to your PostGIS database
    out_name=output_feature_class,  # Name of the feature class in the database
    where_clause="",  
    field_mapping='ID "ID" true true false 4 Long 0 0,First,#,brown_marmorated_stink_bug_mn_XYTableToPoint,ID,-1,-1;'
                  'Observed_Date "Observed Date" true true false 8 DateOnly 0 0,First,#,brown_marmorated_stink_bug_mn_XYTableToPoint,Observed_Date,-1,-1;'
                  'Latitude "Latitude" true true false 8 Double 0 0,First,#,brown_marmorated_stink_bug_mn_XYTableToPoint,Latitude,-1,-1;'
                  'Longitude "Longitude" true true false 8 Double 0 0,First,#,brown_marmorated_stink_bug_mn_XYTableToPoint,Longitude,-1,-1;'
                  'Scientific_Name "Scientific Name" true true false 8000 Text 0 0,First,#,brown_marmorated_stink_bug_mn_XYTableToPoint,Scientific_Name,0,7999;'
                  'Common_Name "Common Name" true true false 8000 Text 0 0,First,#,brown_marmorated_stink_bug_mn_XYTableToPoint,Common_Name,0,7999;'
                  'Image_URL "Image URL" true true false 8000 Text 0 0,First,#,brown_marmorated_stink_bug_mn_XYTableToPoint,Image_URL,0,7999;'
                  'Location_Description "Location Description" true true false 8000 Text 0 0,First,#,brown_marmorated_stink_bug_mn_XYTableToPoint,Location_Description,0,7999;'
                  'Exact_Location_ "Exact Location?" true true false 8000 Text 0 0,First,#,brown_marmorated_stink_bug_mn_XYTableToPoint,Exact_Location_,0,7999;'
                  'Obscured_ "Obscured?" true true false 8000 Text 0 0,First,#,brown_marmorated_stink_bug_mn_XYTableToPoint,Obscured_,0,7999;'
                  'Accuracy__meters_ "Accuracy (meters)" true true false 8 Double 0 0,First,#,brown_marmorated_stink_bug_mn_XYTableToPoint,Accuracy__meters_,-1,-1;'
                  'Count_Observed "Count Observed" true true false 8000 Text 0 0,First,#,brown_marmorated_stink_bug_mn_XYTableToPoint,Count_Observed,0,7999'
)

print("✅ Features successfully exported to PostgreSQL/PostGIS!")

✅ Features successfully exported to PostgreSQL/PostGIS!
